# Downloading a Sentinel-2 geotiff

* **Special requirements:** A Google account, access to Google Earth Engine.
* **Prerequisites:** You should have completed the `2.1_ENGN3903_Satellite images and bands`, the `2.2 ENGN3903_Images, collections, and filters` notebook.


### Load packages

Import Python packages that are used for the analysis.


In [1]:
# %matplotlib inline
import geemap as gmap
import ee

### Connect to Google Earth Engine (GEE)

Connect to the GEE so we can access GEE datasets and computing assets.
You may be required to input your Google account name and password. Please keep those safe and don't share them with anyone.

In [2]:
m = gmap.Map()

***

## Analysis Parameters

In [3]:
# a short date-range around the time you want image for
date_range = '2022-09-01','2022-09-30'

#A geojson that defines the analysis extent
geojson = 'anu_research_forest.geojson'

#the bands from Sentinel-2 that you want
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

#The Projection system to save the data in
crs = 'EPSG:3577'

## Open Geometery defining AOI

In [4]:
geometry = gmap.geojson_to_ee(geojson)

## Load Sentinel 2 image

In [5]:
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return image.updateMask(mask).divide(10000) #re-scale 


s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry)
    .filterDate(date_range[0], date_range[1]) ### Note: you can try different dates
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) #filter out very cloudy images
    .map(maskS2clouds) #map the cloudmasking/rescaling function
    .select(bands)
    .median() #collapse time-dimension using median statistic
    .clip(geometry)
)


## Visualize image

In [6]:
# Lastly, let's give some visualization paramaters to each collection.
s2VisParam = {'bands': ["B4","B3","B2"],
              'max': 0.3,
              'min': 0}


# Let's add the polygon to the map
Map = gmap.Map(center=[-35.30339, 149.06204], zoom=15)              
Map.addLayer(s2, s2VisParam, 's2')
Map.addLayerControl()
Map

Map(center=[-35.30339, 149.06204], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

## Export Image as Geotiff

### Exporting all bands as one file

In [8]:
gmap.ee_export_image(s2, 
                   filename=f'Sentinel-2_{date_range[0]}_{date_range[1]}.tif',
                   scale=10,
                   region=geometry.geometry(),
                   file_per_band=False,
                   crs=crs)

Generating URL ...
Please wait ...
Data downloaded to C:\UserData\burtonc\ENGN3903\engn3903\forward_modelling\Sentinel-2_2022-09-01_2022-09-30.tif


***

## Additional information

**Sources:** 

**License:** The code in this notebook is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle (**check**) site 

**Last modified:** August 2022

***